In [1]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, random_split, Subset, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold

from torchmetrics.classification import Accuracy

In [2]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_transform= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
full_train_dir= "/kaggle/input/chest-xray-pneumonia/chest_xray/train"
full_train= datasets.ImageFolder(root=full_train_dir, transform= train_transform)
full_train_val= datasets.ImageFolder(root=full_train_dir, transform= val_transform)

# full_train.samples returns (image, label) so s[1] imples fetching the labels
labels = [s[1] for s in full_train.samples]

test_dir= "/kaggle/input/chest-xray-pneumonia/chest_xray/test"
test_set= datasets.ImageFolder(root=test_dir, transform= test_transform)

In [5]:
# Treating class imbalance
class_counts= torch.tensor([labels.count(0), labels.count(1)], dtype= torch.float)
class_weights= 1./class_counts
sample_weights= class_weights[[label for _, label in full_train.samples]]


print(class_counts, class_weights, sample_weights)

# Dataset is imbalanced in 1:3 ratio

tensor([1341., 3875.]) tensor([0.0007, 0.0003]) tensor([0.0007, 0.0007, 0.0007,  ..., 0.0003, 0.0003, 0.0003])


In [6]:
model= models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc= nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512,2)
)
model=model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 158MB/s]


In [7]:
def train(dataloader, model, loss_function, optimizer):
    model.train()
    total_loss=0

    for batch, (image, label) in enumerate(dataloader):
        image, label= image.to(device), label.to(device)
        prediction= model(image)
        loss= loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss+= loss
    avg_loss= total_loss/len(dataloader)
    print(f"Training Average Loss: {avg_loss:.4f}")

    
        

In [8]:
val_accuracy= Accuracy(task="multiclass", num_classes=2).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss=0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image, label= image.to(device), label.to(device)
            prediction= model(image)
            loss= loss_function(prediction, label)
            total_loss+= loss

            val_accuracy.update(prediction, label)
            
        avg_loss= total_loss/ len(dataloader)
        accuracy= val_accuracy.compute()*100
        print(f"Validation Loss: {avg_loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.2f}")

    return accuracy

In [9]:
test_accuracy= Accuracy(task="multiclass", num_classes=2).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss=0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image= image.to(device)  # shape: (batch_size, C, H, W)
            label= label.to(device)  # shape: (batch_size,)
            prediction= model(image)
            loss= loss_function(prediction, label)
            total_loss+= loss

            test_accuracy.update(prediction, label)

        avg_loss= total_loss/ len(dataloader)
        accuracy= test_accuracy.compute()*100
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}")

    return accuracy

In [10]:
epochs=15

skf= StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(full_train.samples, labels)):
    print(f"\n--------------Fold {fold+1}------------")
    print(f"Train size: {len(train_idx)}, Val size: {len(val_idx)}")

    train_subset= Subset(full_train, train_idx)
    val_subset= Subset(full_train_val, val_idx)

    #Creating train sampler
    train_sampler= WeightedRandomSampler(
        weights=sample_weights[train_idx],
        num_samples=len(train_idx),
        replacement=True
    )

    
    train_loader= DataLoader(train_subset, sampler= train_sampler, batch_size=64, shuffle=False, num_workers=2) # shuffle has to be false!!
    val_loader= DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)


    model= models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc= nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(512,2)
    )
    model=model.to(device)
    
    loss_function= nn.CrossEntropyLoss()
    optimizer= torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler= torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch= len(train_loader), epochs= epochs)

    best_val_acc= 0.0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        train(train_loader, model, loss_function, optimizer)
        val_acc= validate(val_loader, model, loss_function)

        if val_acc > best_val_acc:
            best_val_acc= val_acc
    print(f"Best Validation Accuracy in Fold {fold+1}: {best_val_acc:.2f}%")
    print(f"-----------------------------------------------------------------")


print("\n")
test_loader= DataLoader(test_set, batch_size=64, shuffle=False, num_workers=2)
test(test_loader, model, loss_function)


--------------Fold 1------------
Train size: 4172, Val size: 1044
Epoch 1/15
Training Average Loss: 0.1337
Validation Loss: 0.8128
Validation Accuracy: 83.72
Epoch 2/15
Training Average Loss: 0.1286
Validation Loss: 6.0222
Validation Accuracy: 45.59
Epoch 3/15
Training Average Loss: 0.2717
Validation Loss: 157.4154
Validation Accuracy: 26.15
Epoch 4/15
Training Average Loss: 0.2740
Validation Loss: 0.9770
Validation Accuracy: 65.52
Epoch 5/15
Training Average Loss: 0.1966
Validation Loss: 0.3845
Validation Accuracy: 85.82
Epoch 6/15
Training Average Loss: 0.1880
Validation Loss: 0.3475
Validation Accuracy: 83.33
Epoch 7/15
Training Average Loss: 0.1911
Validation Loss: 0.6451
Validation Accuracy: 75.67
Epoch 8/15
Training Average Loss: 0.1849
Validation Loss: 0.1349
Validation Accuracy: 94.35
Epoch 9/15
Training Average Loss: 0.1522
Validation Loss: 0.1209
Validation Accuracy: 96.07
Epoch 10/15
Training Average Loss: 0.1431
Validation Loss: 1.0282
Validation Accuracy: 59.39
Epoch 11/1

tensor(81.8910, device='cuda:0')